In [ ]:
class ImageModerationSystem:
    """
    Consolidated image moderation system implementing Phase 1 & 2
    
    This is a foundational implementation - components can be used 
    independently or combined based on needs. This is just Phase 1 & 2
    out of a much larger 7-phase system.
    """
    
    def __init__(self, config: Dict[str, Any] = None):
        self.config = config or {}
        
        # Phase 1 Components
        self.store = HashStore(self.config.get('db_path', 'moderation.db'))
        self.mih_index = MIHIndex(self.store)
        self.hma_engine = None  # Initialized when needed
        self.csam_pipeline = None  # Initialized when needed
        
        # Phase 2 Components  
        self.nsfw_detector = NSFWDetector()
        self.violence_detector = ViolenceGoreDetector()
        self.weapon_detector = WeaponDetector()
        self.hate_symbol_detector = HateSymbolDetector()
        
        # System state
        self.initialized = False
    
    def initialize(self):
        """Initialize all system components"""
        if self.initialized:
            return
            
        try:
            # Initialize storage
            self.store.init()
            
            # Build MIH index if data exists
            if any(True for _ in self.store.iter_all_hashes()):
                self.mih_index.build()
            
            print("Image moderation system initialized successfully")
            self.initialized = True
            
        except Exception as e:
            logging.error(f"Error initializing moderation system: {e}")
            raise
    
    def analyze_image_complete(self, image_path: str, media_id: str = None, 
                              surface_type: str = 'feed') -> Dict[str, Any]:
        """
        Run complete analysis on an image using all available components
        This is a basic example - production would have more sophisticated
        orchestration and decision logic
        """
        if not self.initialized:
            self.initialize()
            
        if not media_id:
            media_id = f"img_{int(time.time())}"
        
        results = {
            'media_id': media_id,
            'image_path': image_path,
            'surface_type': surface_type,
            'timestamp': datetime.now().isoformat(),
            'phase1_results': {},
            'phase2_results': {},
            'final_action': 'allow',
            'confidence': 0.0
        }
        
        try:
            # Phase 1: Hash-based matching
            hash_hex, quality = PDQHasher.compute_pdq(image_path)
            matches = self.mih_index.query(hash_hex, max_distance=30)
            
            results['phase1_results'] = {
                'hash': hash_hex,
                'quality': quality,
                'matches': matches,
                'match_count': len(matches)
            }
            
            # If hash matches found, that takes priority
            if matches:
                results['final_action'] = 'block'
                results['confidence'] = 1.0
                return results
            
            # Phase 2: ML-based classification
            # Note: In production, these might run in parallel
            
            # NSFW Detection
            nsfw_result = self.nsfw_detector.analyze_image(image_path, surface_type)
            results['phase2_results']['nsfw'] = nsfw_result
            
            # Violence/Gore Detection
            violence_result = self.violence_detector.analyze_image(image_path)
            results['phase2_results']['violence'] = violence_result
            
            # Weapon Detection
            weapon_result = self.weapon_detector.detect_weapons(image_path)
            results['phase2_results']['weapons'] = weapon_result
            
            # Hate Symbol Detection
            hate_result = self.hate_symbol_detector.analyze_image(image_path)
            results['phase2_results']['hate_symbols'] = hate_result
            
            # Simple decision logic (production would be much more sophisticated)
            if (nsfw_result.get('action') == 'block' or 
                violence_result.get('overall_action') == 'block' or
                hate_result.get('overall_action') == 'block' or
                weapon_result.get('overall_confidence', 0) > 0.8):
                
                results['final_action'] = 'block'
                results['confidence'] = max(
                    nsfw_result.get('nsfw_score', 0),
                    violence_result.get('max_violence_score', 0),
                    hate_result.get('confidence', 0),
                    weapon_result.get('overall_confidence', 0)
                )
            
            return results
            
        except Exception as e:
            logging.error(f"Error analyzing image {media_id}: {e}")
            results['error'] = str(e)
            results['final_action'] = 'allow'  # Fail open
            return results

# System usage examples and integration notes
print("""
Image Moderation System - Phase 1 & 2 Implementation Complete

This notebook contains implementations for:

PHASE 1 - Hash Matching "Seatbelts":
- 1A: PDQ Perceptual Hashing with Multi-Index Hashing
- 1B: ThreatExchange/HMA Integration Framework  
- 1C: CSAM/NCII Detection Pipelines (PhotoDNA, CSAI Match, StopNCII)

PHASE 2 - Core Classifiers:
- 2A: NSFW/Nudity Detection (OpenNSFW2 + CLIP ensemble)
- 2B: Violence/Gore & Weapons Detection
- 2C: Hate Symbols & Extremist Content Detection

INTEGRATION NOTES:
- Each component can be used independently
- Database schema supports all features
- Fail-safes and queuing built into hash matching
- Configurable thresholds per surface type
- Basic orchestration provided in ImageModerationSystem class

NEXT PHASES (not implemented here):
- Phase 3: OCR & Meme Understanding  
- Phase 4: Teen Safety & Age Gates
- Phase 5: Policy Engine & Actions
- Phase 6: Privacy & Compliance  
- Phase 7: Scale & Adversarial Testing

This is foundational work - production deployment would require
additional infrastructure, monitoring, and safety measures.
""")

# Integrated Moderation System

Complete image moderation pipeline combining all Phase 1 and Phase 2 components.

In [ ]:
class HateSymbolDetector:
    """Detection of hate symbols and extremist logos"""
    
    def __init__(self):
        self.models_loaded = False
        self.clip_model = None
        self.clip_processor = None
        self.ocr_reader = None
        self.symbol_database = HateSymbolDatabase()
        
        # Confidence thresholds
        self.thresholds = {
            'symbol_detection': 0.7,
            'text_detection': 0.8,
            'combined_confidence': 0.6  # Lower when both symbol and text agree
        }
    
    def load_models(self):
        """Load hate symbol detection models"""
        try:
            # Load CLIP model
            if not self.clip_model:
                self.clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
                self.clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
            
            # Load OCR model
            if not self.ocr_reader:
                self.ocr_reader = easyocr.Reader(['en'])
            
            self.models_loaded = True
            print("Hate symbol detection models loaded successfully")
            
        except Exception as e:
            logging.error(f"Error loading hate symbol detection models: {e}")
    
    def detect_symbols_with_clip(self, image: Image.Image) -> Dict[str, Any]:
        """Detect hate symbols using CLIP"""
        if not self.clip_model:
            return {}
        
        try:
            # Get known hate symbols and extremist logos
            symbol_prompts = self.symbol_database.get_symbol_prompts()
            safe_prompts = [
                "normal logo", "safe symbol", "regular image", 
                "appropriate content", "brand logo"
            ]
            
            results = {}
            
            for category, prompts in symbol_prompts.items():
                all_prompts = prompts + safe_prompts
                
                inputs = self.clip_processor(
                    text=all_prompts,
                    images=image,
                    return_tensors="pt",
                    padding=True
                )
                
                with torch.no_grad():
                    outputs = self.clip_model(**inputs)
                    logits_per_image = outputs.logits_per_image
                    probs = logits_per_image.softmax(dim=1)
                    
                    # Get maximum confidence for hate symbol prompts
                    symbol_probs = probs[0][:len(prompts)]
                    max_symbol_prob = symbol_probs.max().item()
                    best_match_idx = symbol_probs.argmax().item()
                    
                    safe_prob = probs[0][len(prompts):].mean().item()
                    
                    # Normalize confidence
                    total_prob = max_symbol_prob + safe_prob
                    if total_prob > 0:
                        confidence = max_symbol_prob / total_prob
                    else:
                        confidence = 0.0
                    
                    if confidence > self.thresholds['symbol_detection']:
                        results[category] = {
                            'confidence': confidence,
                            'matched_symbol': prompts[best_match_idx],
                            'detection_method': 'clip'
                        }
            
            return results
            
        except Exception as e:
            logging.error(f"CLIP symbol detection error: {e}")
            return {}
    
    def extract_text_with_ocr(self, image_path: str) -> List[Dict[str, Any]]:
        """Extract text from image using OCR"""
        if not self.ocr_reader:
            return []
        
        try:
            results = self.ocr_reader.readtext(image_path)
            
            extracted_text = []
            for (bbox, text, confidence) in results:
                if confidence > 0.5:  # Filter low confidence OCR results
                    extracted_text.append({
                        'text': text.strip(),
                        'confidence': confidence,
                        'bbox': bbox
                    })
            
            return extracted_text
            
        except Exception as e:
            logging.error(f"OCR extraction error: {e}")
            return []
    
    def analyze_text_for_hate(self, text_results: List[Dict[str, Any]]) -> Dict[str, Any]:
        """Analyze extracted text for hate speech and extremist content"""
        hate_keywords = self.symbol_database.get_hate_keywords()
        
        detected_hate_text = []
        max_confidence = 0.0
        
        for text_item in text_results:
            text_lower = text_item['text'].lower()
            
            for category, keywords in hate_keywords.items():
                for keyword in keywords:
                    if keyword.lower() in text_lower:
                        confidence = text_item['confidence'] * 0.9  # Slight penalty for text matching
                        
                        detected_hate_text.append({
                            'category': category,
                            'matched_keyword': keyword,
                            'full_text': text_item['text'],
                            'confidence': confidence,
                            'bbox': text_item['bbox'],
                            'detection_method': 'ocr'
                        })
                        
                        max_confidence = max(max_confidence, confidence)
        
        return {
            'hate_text_detected': detected_hate_text,
            'max_confidence': max_confidence,
            'total_matches': len(detected_hate_text)
        }
    
    def analyze_image(self, image_path: str) -> Dict[str, Any]:
        """Comprehensive hate symbol and extremist content analysis"""
        if not self.models_loaded:
            self.load_models()
        
        try:
            # Load image
            image_pil = Image.open(image_path).convert('RGB')
            
            # Symbol detection with CLIP
            symbol_results = self.detect_symbols_with_clip(image_pil)
            
            # Text extraction and analysis
            text_results = self.extract_text_with_ocr(image_path)
            hate_text_analysis = self.analyze_text_for_hate(text_results)
            
            # Combined analysis
            final_result = {
                'symbol_detections': symbol_results,
                'text_detections': hate_text_analysis,
                'all_extracted_text': [item['text'] for item in text_results],
                'overall_action': 'allow',
                'confidence': 0.0,
                'detection_methods': []
            }
            
            # Determine final action based on combined evidence
            symbol_confidence = max([r['confidence'] for r in symbol_results.values()], default=0.0)
            text_confidence = hate_text_analysis['max_confidence']
            
            # If both symbol and text detection agree, lower the threshold
            if symbol_results and hate_text_analysis['hate_text_detected']:
                combined_confidence = (symbol_confidence + text_confidence) / 2
                threshold = self.thresholds['combined_confidence']
                final_result['detection_methods'] = ['symbol', 'text']
            elif symbol_results:
                combined_confidence = symbol_confidence
                threshold = self.thresholds['symbol_detection']
                final_result['detection_methods'] = ['symbol']
            elif hate_text_analysis['hate_text_detected']:
                combined_confidence = text_confidence
                threshold = self.thresholds['text_detection']
                final_result['detection_methods'] = ['text']
            else:
                combined_confidence = 0.0
                threshold = 1.0
            
            final_result['confidence'] = combined_confidence
            final_result['overall_action'] = 'block' if combined_confidence > threshold else 'allow'
            
            return final_result
            
        except Exception as e:
            logging.error(f"Hate symbol analysis error: {e}")
            return {
                'error': str(e),
                'overall_action': 'allow',
                'confidence': 0.0
            }

class HateSymbolDatabase:
    """Database of known hate symbols, logos, and keywords"""
    
    def __init__(self):
        self.symbol_categories = self._load_symbol_database()
        self.hate_keywords = self._load_hate_keywords()
    
    def _load_symbol_database(self) -> Dict[str, List[str]]:
        """Load database of hate symbols and extremist logos"""
        # This would normally be loaded from a curated database
        return {
            'nazi_symbols': [
                'swastika', 'nazi symbol', 'third reich emblem',
                'ss symbol', 'hitler youth symbol', 'iron cross'
            ],
            'white_supremacist': [
                'confederate flag', 'kkk symbol', 'white power symbol',
                'celtic cross', 'blood and honor', 'fourteen words'
            ],
            'extremist_groups': [
                'isis flag', 'terrorist logo', 'extremist symbol',
                'militia patch', 'hate group logo', 'radical emblem'
            ],
            'gang_symbols': [
                'gang tag', 'gang symbol', 'criminal organization logo',
                'cartel symbol', 'prison gang mark'
            ]
        }
    
    def _load_hate_keywords(self) -> Dict[str, List[str]]:
        """Load database of hate keywords and phrases"""
        # Simplified example - production would use comprehensive, reviewed database
        return {
            'racial_slurs': [
                # Would contain actual slurs but omitted here for safety
                'hate_keyword_1', 'hate_keyword_2'
            ],
            'extremist_phrases': [
                'white power', 'blood and soil', 'race war',
                'day of the rope', 'turner diaries'
            ],
            'terrorist_language': [
                'allahu akbar', 'death to america', 'jihad',
                'martyrdom', 'caliphate'
            ],
            'holocaust_denial': [
                'holohoax', 'six million lie', 'gas chamber myth',
                'holocaust denial', 'revisionist history'
            ]
        }
    
    def get_symbol_prompts(self) -> Dict[str, List[str]]:
        """Get CLIP prompts for symbol detection"""
        return self.symbol_categories
    
    def get_hate_keywords(self) -> Dict[str, List[str]]:
        """Get hate keywords for text analysis"""
        return self.hate_keywords
    
    def add_symbol_category(self, category: str, prompts: List[str]):
        """Add new symbol category (for updates)"""
        self.symbol_categories[category] = prompts
    
    def add_hate_keywords(self, category: str, keywords: List[str]):
        """Add new hate keywords (for updates)"""
        if category in self.hate_keywords:
            self.hate_keywords[category].extend(keywords)
        else:
            self.hate_keywords[category] = keywords
    
    def update_from_external_source(self, source_url: str):
        """Update database from external hate symbol database"""
        # Placeholder for integration with external databases like ADL's database
        try:
            # Would implement API calls to external databases
            logging.info(f"Would update symbols from {source_url}")
            pass
        except Exception as e:
            logging.error(f"Error updating from external source: {e}")

print("Hate symbols and extremist content detection system initialized successfully")

# Phase 2C: Hate Symbols & Extremist Logos Detection

Symbol + text fusion to detect hateful propaganda and extremist branding.

In [ ]:
class ViolenceGoreDetector:
    """Detection of violent and graphic content"""
    
    def __init__(self):
        self.models_loaded = False
        self.clip_model = None
        self.clip_processor = None
        self.color_analyzer = ColorAnalyzer()
        
        # Violence categories and thresholds
        self.violence_categories = {
            'blood': 0.7,
            'injury': 0.8,
            'corpse': 0.9,
            'weapons': 0.8,
            'fighting': 0.75
        }
    
    def load_models(self):
        """Load violence detection models"""
        try:
            if not self.clip_model:
                self.clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
                self.clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
            
            self.models_loaded = True
            print("Violence detection models loaded successfully")
            
        except Exception as e:
            logging.error(f"Error loading violence detection models: {e}")
    
    def analyze_violence_with_clip(self, image: Image.Image) -> Dict[str, float]:
        """Analyze violence using CLIP"""
        if not self.clip_model:
            return {}
        
        try:
            violence_prompts = {
                'blood': [
                    "blood", "bloody scene", "red liquid", "bleeding",
                    "bloodstain", "gore", "graphic blood"
                ],
                'injury': [
                    "injury", "wound", "hurt person", "medical emergency",
                    "accident victim", "injured body", "trauma"
                ],
                'corpse': [
                    "dead body", "corpse", "deceased person", "death scene",
                    "morgue", "funeral", "dead animal"
                ],
                'weapons': [
                    "gun", "knife", "weapon", "firearm", "blade",
                    "rifle", "pistol", "sword", "machete"
                ],
                'fighting': [
                    "fighting", "violence", "attack", "assault",
                    "brawl", "combat", "aggression"
                ]
            }
            
            safe_prompts = [
                "safe image", "normal photo", "peaceful scene",
                "family friendly content", "appropriate image"
            ]
            
            results = {}
            
            for category, prompts in violence_prompts.items():
                all_prompts = prompts + safe_prompts
                
                inputs = self.clip_processor(
                    text=all_prompts,
                    images=image,
                    return_tensors="pt",
                    padding=True
                )
                
                with torch.no_grad():
                    outputs = self.clip_model(**inputs)
                    logits_per_image = outputs.logits_per_image
                    probs = logits_per_image.softmax(dim=1)
                    
                    # Average probability for violence prompts vs safe prompts
                    violence_prob = probs[0][:len(prompts)].mean().item()
                    safe_prob = probs[0][len(prompts):].mean().item()
                    
                    # Normalize
                    total_prob = violence_prob + safe_prob
                    if total_prob > 0:
                        violence_score = violence_prob / total_prob
                    else:
                        violence_score = 0.0
                    
                    results[category] = violence_score
            
            return results
            
        except Exception as e:
            logging.error(f"CLIP violence analysis error: {e}")
            return {}
    
    def analyze_color_patterns(self, image: np.ndarray) -> Dict[str, float]:
        """Analyze color patterns indicative of violence/gore"""
        return self.color_analyzer.analyze_for_violence(image)
    
    def analyze_image(self, image_path: str) -> Dict[str, Any]:
        """Comprehensive violence/gore analysis"""
        if not self.models_loaded:
            self.load_models()
        
        try:
            # Load image
            image_pil = Image.open(image_path).convert('RGB')
            image_cv = cv2.imread(image_path)
            image_cv_rgb = cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB)
            
            # CLIP-based analysis
            clip_results = self.analyze_violence_with_clip(image_pil)
            
            # Color pattern analysis
            color_results = self.analyze_color_patterns(image_cv_rgb)
            
            # Combine results
            final_scores = {}
            actions = {}
            
            for category in self.violence_categories:
                clip_score = clip_results.get(category, 0.0)
                color_score = color_results.get(category, 0.0)
                
                # Weighted combination
                combined_score = (clip_score * 0.7 + color_score * 0.3)
                final_scores[category] = combined_score
                
                # Action decision
                threshold = self.violence_categories[category]
                actions[category] = 'block' if combined_score > threshold else 'allow'
            
            # Overall action (block if any category triggers)
            overall_action = 'block' if 'block' in actions.values() else 'allow'
            max_score = max(final_scores.values()) if final_scores else 0.0
            
            return {
                'violence_scores': final_scores,
                'clip_results': clip_results,
                'color_results': color_results,
                'category_actions': actions,
                'overall_action': overall_action,
                'max_violence_score': max_score,
                'graphicness': self._calculate_graphicness(final_scores)
            }
            
        except Exception as e:
            logging.error(f"Violence analysis error: {e}")
            return {
                'error': str(e),
                'overall_action': 'allow',
                'violence_scores': {}
            }
    
    def _calculate_graphicness(self, scores: Dict[str, float]) -> float:
        """Calculate overall graphicness score"""
        if not scores:
            return 0.0
        
        # Weight different categories for graphicness
        weights = {
            'blood': 0.3,
            'injury': 0.2,
            'corpse': 0.4,
            'weapons': 0.05,
            'fighting': 0.05
        }
        
        weighted_sum = sum(scores.get(cat, 0) * weight 
                          for cat, weight in weights.items())
        return min(1.0, weighted_sum)

class WeaponDetector:
    """Specialized weapon detection"""
    
    def __init__(self):
        self.models_loaded = False
        self.clip_model = None
        self.clip_processor = None
        self.shape_detector = WeaponShapeDetector()
        
        self.weapon_types = {
            'firearm': ['gun', 'pistol', 'rifle', 'shotgun', 'revolver'],
            'blade': ['knife', 'sword', 'machete', 'dagger', 'blade'],
            'blunt': ['bat', 'club', 'hammer', 'pipe', 'stick']
        }
    
    def load_models(self):
        """Load weapon detection models"""
        try:
            if not self.clip_model:
                self.clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
                self.clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
            
            self.models_loaded = True
            print("Weapon detection models loaded successfully")
            
        except Exception as e:
            logging.error(f"Error loading weapon detection models: {e}")
    
    def detect_weapons(self, image_path: str) -> Dict[str, Any]:
        """Detect weapons in image"""
        if not self.models_loaded:
            self.load_models()
        
        try:
            image_pil = Image.open(image_path).convert('RGB')
            image_cv = cv2.imread(image_path)
            
            results = {
                'weapons_detected': [],
                'confidence_scores': {},
                'shape_analysis': {},
                'overall_confidence': 0.0
            }
            
            # CLIP-based weapon detection
            for weapon_type, weapon_names in self.weapon_types.items():
                all_prompts = weapon_names + ['safe object', 'household item', 'tool']
                
                inputs = self.clip_processor(
                    text=all_prompts,
                    images=image_pil,
                    return_tensors="pt",
                    padding=True
                )
                
                with torch.no_grad():
                    outputs = self.clip_model(**inputs)
                    logits_per_image = outputs.logits_per_image
                    probs = logits_per_image.softmax(dim=1)
                    
                    weapon_prob = probs[0][:len(weapon_names)].max().item()
                    results['confidence_scores'][weapon_type] = weapon_prob
                    
                    if weapon_prob > 0.7:  # High confidence threshold
                        results['weapons_detected'].append(weapon_type)
            
            # Shape-based detection
            shape_results = self.shape_detector.detect_weapon_shapes(image_cv)
            results['shape_analysis'] = shape_results
            
            # Overall confidence
            if results['confidence_scores']:
                results['overall_confidence'] = max(results['confidence_scores'].values())
            
            return results
            
        except Exception as e:
            logging.error(f"Weapon detection error: {e}")
            return {'error': str(e), 'weapons_detected': []}

class ColorAnalyzer:
    """Color pattern analysis for violence detection"""
    
    def analyze_for_violence(self, image: np.ndarray) -> Dict[str, float]:
        """Analyze color patterns indicative of violence"""
        try:
            hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            
            results = {
                'blood': self._detect_blood_colors(hsv_image),
                'injury': self._detect_injury_colors(hsv_image),
                'corpse': self._detect_corpse_colors(hsv_image)
            }
            
            return results
            
        except Exception as e:
            logging.error(f"Color analysis error: {e}")
            return {'blood': 0.0, 'injury': 0.0, 'corpse': 0.0}
    
    def _detect_blood_colors(self, hsv_image: np.ndarray) -> float:
        """Detect blood-like red colors"""
        # Define HSV ranges for blood-like colors
        blood_lower = np.array([0, 50, 50])
        blood_upper = np.array([10, 255, 200])
        
        blood_mask = cv2.inRange(hsv_image, blood_lower, blood_upper)
        blood_pixels = np.sum(blood_mask > 0)
        total_pixels = hsv_image.shape[0] * hsv_image.shape[1]
        
        blood_ratio = blood_pixels / total_pixels
        return min(1.0, blood_ratio * 10)  # Scale factor
    
    def _detect_injury_colors(self, hsv_image: np.ndarray) -> float:
        """Detect colors associated with injuries (bruises, etc.)"""
        # Purple/blue ranges for bruising
        bruise_lower = np.array([120, 30, 30])
        bruise_upper = np.array([150, 255, 150])
        
        bruise_mask = cv2.inRange(hsv_image, bruise_lower, bruise_upper)
        bruise_pixels = np.sum(bruise_mask > 0)
        total_pixels = hsv_image.shape[0] * hsv_image.shape[1]
        
        bruise_ratio = bruise_pixels / total_pixels
        return min(1.0, bruise_ratio * 8)
    
    def _detect_corpse_colors(self, hsv_image: np.ndarray) -> float:
        """Detect colors associated with death (pale, grey tones)"""
        # Grey/pale color ranges
        pale_lower = np.array([0, 0, 40])
        pale_upper = np.array([180, 30, 120])
        
        pale_mask = cv2.inRange(hsv_image, pale_lower, pale_upper)
        pale_pixels = np.sum(pale_mask > 0)
        total_pixels = hsv_image.shape[0] * hsv_image.shape[1]
        
        pale_ratio = pale_pixels / total_pixels
        return min(1.0, pale_ratio * 5)

class WeaponShapeDetector:
    """Shape-based weapon detection using basic computer vision"""
    
    def detect_weapon_shapes(self, image: np.ndarray) -> Dict[str, Any]:
        """Detect weapon-like shapes"""
        try:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            edges = cv2.Canny(gray, 50, 150)
            
            # Find contours
            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            weapon_shapes = []
            
            for contour in contours:
                if cv2.contourArea(contour) > 1000:  # Filter small shapes
                    # Analyze shape characteristics
                    x, y, w, h = cv2.boundingRect(contour)
                    aspect_ratio = w / h if h > 0 else 0
                    
                    # Simple heuristics for weapon-like shapes
                    if self._is_gun_like_shape(contour, aspect_ratio):
                        weapon_shapes.append({
                            'type': 'gun_like',
                            'bbox': [x, y, w, h],
                            'confidence': 0.6  # Conservative confidence
                        })
                    elif self._is_knife_like_shape(contour, aspect_ratio):
                        weapon_shapes.append({
                            'type': 'blade_like',
                            'bbox': [x, y, w, h],
                            'confidence': 0.5
                        })
            
            return {
                'detected_shapes': weapon_shapes,
                'shape_count': len(weapon_shapes)
            }
            
        except Exception as e:
            logging.error(f"Shape detection error: {e}")
            return {'detected_shapes': [], 'shape_count': 0}
    
    def _is_gun_like_shape(self, contour: np.ndarray, aspect_ratio: float) -> bool:
        """Check if contour resembles a gun shape"""
        # Very basic heuristics - would need ML model for production
        if 2.0 < aspect_ratio < 6.0:  # Elongated shape
            hull = cv2.convexHull(contour)
            hull_area = cv2.contourArea(hull)
            contour_area = cv2.contourArea(contour)
            
            if hull_area > 0:
                solidity = contour_area / hull_area
                return 0.7 < solidity < 0.95  # Relatively solid but with some concavity
        
        return False
    
    def _is_knife_like_shape(self, contour: np.ndarray, aspect_ratio: float) -> bool:
        """Check if contour resembles a knife/blade shape"""
        if 3.0 < aspect_ratio < 10.0:  # Very elongated
            # Check for pointed end (simplified)
            hull = cv2.convexHull(contour)
            hull_area = cv2.contourArea(hull)
            contour_area = cv2.contourArea(contour)
            
            if hull_area > 0:
                solidity = contour_area / hull_area
                return solidity > 0.8  # Should be quite solid for a blade
        
        return False

print("Violence/gore and weapons detection system initialized successfully")

# Phase 2B: Violence/Gore and Weapons Detection

Multi-label classifier for graphic content and weapon detection.

In [ ]:
class NSFWDetector:
    """NSFW detection using ensemble of models"""
    
    def __init__(self):
        self.models_loaded = False
        self.opennsfw_model = None
        self.clip_model = None
        self.clip_processor = None
        self.region_detector = None
        
        # Thresholds per surface type
        self.thresholds = {
            'feed': 0.8,
            'avatar': 0.9,
            'dm': 0.7
        }
    
    def load_models(self):
        """Load NSFW detection models"""
        try:
            # Load OpenNSFW2 (if available)
            try:
                import opennsfw2 as n2
                self.opennsfw_model = n2
                print("OpenNSFW2 loaded")
            except ImportError:
                print("OpenNSFW2 not available, installing...")
                os.system("pip install opennsfw2")
                import opennsfw2 as n2
                self.opennsfw_model = n2
            
            # Load CLIP model for NSFW detection
            self.clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
            self.clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
            
            # Create simple region detector using basic CV
            self.region_detector = RegionDetector()
            
            self.models_loaded = True
            print("NSFW detection models loaded successfully")
            
        except Exception as e:
            logging.error(f"Error loading NSFW models: {e}")
    
    def analyze_with_opennsfw(self, image_path: str) -> float:
        """Analyze image with OpenNSFW2"""
        if not self.opennsfw_model:
            return 0.0
            
        try:
            nsfw_probability = self.opennsfw_model.predict_image(image_path)
            return float(nsfw_probability)
        except Exception as e:
            logging.error(f"OpenNSFW2 error: {e}")
            return 0.0
    
    def analyze_with_clip(self, image: Image.Image) -> float:
        """Analyze image with CLIP-based NSFW classifier"""
        if not self.clip_model:
            return 0.0
            
        try:
            # Define NSFW-related text prompts
            nsfw_prompts = [
                "explicit sexual content",
                "nudity",
                "pornographic image",
                "adult content",
                "sexual activity"
            ]
            
            safe_prompts = [
                "safe for work image",
                "normal photo",
                "appropriate content",
                "family friendly image"
            ]
            
            inputs = self.clip_processor(
                text=nsfw_prompts + safe_prompts,
                images=image,
                return_tensors="pt",
                padding=True
            )
            
            with torch.no_grad():
                outputs = self.clip_model(**inputs)
                logits_per_image = outputs.logits_per_image
                probs = logits_per_image.softmax(dim=1)
                
                # Average NSFW probability
                nsfw_prob = probs[0][:len(nsfw_prompts)].mean().item()
                return nsfw_prob
                
        except Exception as e:
            logging.error(f"CLIP NSFW error: {e}")
            return 0.0
    
    def detect_regions(self, image: np.ndarray) -> Dict[str, Any]:
        """Detect specific body regions"""
        if not self.region_detector:
            return {'regions': [], 'confidence': 0.0}
            
        return self.region_detector.detect_regions(image)
    
    def analyze_image(self, image_path: str, surface_type: str = 'feed') -> Dict[str, Any]:
        """Comprehensive NSFW analysis"""
        if not self.models_loaded:
            self.load_models()
        
        try:
            # Load image
            image_pil = Image.open(image_path).convert('RGB')
            image_cv = cv2.imread(image_path)
            image_cv_rgb = cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB)
            
            # OpenNSFW2 analysis
            opennsfw_score = self.analyze_with_opennsfw(image_path)
            
            # CLIP analysis
            clip_score = self.analyze_with_clip(image_pil)
            
            # Region detection
            regions_result = self.detect_regions(image_cv_rgb)
            
            # Ensemble scoring
            ensemble_score = (opennsfw_score * 0.6 + clip_score * 0.4)
            
            # Apply region weighting
            region_penalty = regions_result['confidence'] * 0.2
            final_score = min(1.0, ensemble_score + region_penalty)
            
            # Threshold decision
            threshold = self.thresholds.get(surface_type, 0.8)
            action = 'block' if final_score > threshold else 'allow'
            
            return {
                'nsfw_score': final_score,
                'opennsfw_score': opennsfw_score,
                'clip_score': clip_score,
                'regions': regions_result['regions'],
                'region_confidence': regions_result['confidence'],
                'threshold': threshold,
                'action': action,
                'surface_type': surface_type
            }
            
        except Exception as e:
            logging.error(f"NSFW analysis error: {e}")
            return {
                'error': str(e),
                'action': 'allow',  # Fail open for safety
                'nsfw_score': 0.0
            }

class RegionDetector:
    """Simple region detector for body parts"""
    
    def __init__(self):
        # Load basic classifiers for skin detection
        self.skin_detector = self._create_skin_detector()
    
    def _create_skin_detector(self):
        """Create simple skin color detector"""
        # HSV ranges for skin color detection
        return {
            'lower': np.array([0, 20, 70], dtype=np.uint8),
            'upper': np.array([20, 255, 255], dtype=np.uint8)
        }
    
    def detect_skin_regions(self, image: np.ndarray) -> np.ndarray:
        """Detect skin-colored regions"""
        hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        
        skin_mask = cv2.inRange(
            hsv,
            self.skin_detector['lower'],
            self.skin_detector['upper']
        )
        
        # Morphological operations to clean up mask
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        skin_mask = cv2.morphologyEx(skin_mask, cv2.MORPH_OPEN, kernel)
        skin_mask = cv2.morphologyEx(skin_mask, cv2.MORPH_CLOSE, kernel)
        
        return skin_mask
    
    def detect_regions(self, image: np.ndarray) -> Dict[str, Any]:
        """Detect and classify body regions"""
        try:
            skin_mask = self.detect_skin_regions(image)
            
            # Find contours
            contours, _ = cv2.findContours(
                skin_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
            )
            
            regions = []
            total_skin_area = 0
            
            for contour in contours:
                area = cv2.contourArea(contour)
                if area > 1000:  # Filter small regions
                    x, y, w, h = cv2.boundingRect(contour)
                    
                    # Simple heuristics for region classification
                    aspect_ratio = w / h if h > 0 else 0
                    relative_area = area / (image.shape[0] * image.shape[1])
                    
                    region_type = self._classify_region(
                        x, y, w, h, aspect_ratio, relative_area, image.shape
                    )
                    
                    regions.append({
                        'type': region_type,
                        'bbox': [x, y, w, h],
                        'area': area,
                        'confidence': min(1.0, area / 10000)
                    })
                    
                    total_skin_area += area
            
            # Overall confidence based on skin area ratio
            skin_ratio = total_skin_area / (image.shape[0] * image.shape[1])
            confidence = min(1.0, skin_ratio * 3)  # Scale factor
            
            return {
                'regions': regions,
                'confidence': confidence,
                'total_skin_area': total_skin_area,
                'skin_ratio': skin_ratio
            }
            
        except Exception as e:
            logging.error(f"Region detection error: {e}")
            return {'regions': [], 'confidence': 0.0}
    
    def _classify_region(self, x: int, y: int, w: int, h: int, 
                        aspect_ratio: float, relative_area: float, 
                        image_shape: Tuple[int, int, int]) -> str:
        """Simple heuristic region classification"""
        img_h, img_w = image_shape[:2]
        
        # Normalize coordinates
        center_x = (x + w/2) / img_w
        center_y = (y + h/2) / img_h
        
        # Very simple rules - would need proper ML model for production
        if center_y < 0.3 and relative_area > 0.05:
            return 'torso'
        elif center_y < 0.7 and relative_area > 0.02:
            return 'limb'
        elif relative_area > 0.1:
            return 'large_skin_area'
        else:
            return 'skin_region'

print("NSFW detection system initialized successfully")

# Phase 2A: Nudity/Sexual Content Detector

Ensemble model to distinguish explicit sexual content with fine-grained region detection.

In [ ]:
class PhotoDNAIntegration:
    """Integration with Microsoft PhotoDNA for CSAM detection"""
    
    def __init__(self, api_key: str, endpoint: str):
        self.api_key = api_key
        self.endpoint = endpoint
        self.headers = {
            'Ocp-Apim-Subscription-Key': api_key,
            'Content-Type': 'application/octet-stream'
        }
    
    def analyze_image(self, image_bytes: bytes) -> Dict[str, Any]:
        """Analyze image using PhotoDNA"""
        try:
            response = requests.post(
                f"{self.endpoint}/photodna/v1.0/Match",
                headers=self.headers,
                data=image_bytes
            )
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            logging.error(f"PhotoDNA API error: {e}")
            return {"error": str(e)}

class CSAIMatchIntegration:
    """Integration with Google CSAI Match"""
    
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "https://safesearch.googleapis.com/v1"
    
    def analyze_image(self, image_bytes: bytes) -> Dict[str, Any]:
        """Analyze image using CSAI Match"""
        try:
            import base64
            image_b64 = base64.b64encode(image_bytes).decode('utf-8')
            
            payload = {
                'image': {
                    'content': image_b64
                },
                'features': [
                    {'type': 'SAFE_SEARCH_DETECTION'},
                    {'type': 'OBJECT_LOCALIZATION'}
                ]
            }
            
            response = requests.post(
                f"{self.base_url}/images:annotate?key={self.api_key}",
                json=payload
            )
            response.raise_for_status()
            return response.json()
        except Exception as e:
            logging.error(f"CSAI Match API error: {e}")
            return {"error": str(e)}

class StopNCIIIntegration:
    """StopNCII client-side hashing integration"""
    
    def __init__(self, store: HashStore):
        self.store = store
        self.stopncii_hashes = set()
    
    def load_survivor_hashes(self, hash_file: str):
        """Load hashes submitted by survivors via StopNCII"""
        try:
            with open(hash_file, 'r') as f:
                for line in f:
                    hash_hex = line.strip()
                    if len(hash_hex) == 64:  # Valid PDQ hash
                        self.stopncii_hashes.add(hash_hex)
                        
                        media_id = f"stopncii_{hashlib.md5(hash_hex.encode()).hexdigest()[:16]}"
                        self.store.upsert_media_hash(
                            media_id=media_id,
                            hash_hex=hash_hex,
                            quality=100,
                            source="stopncii:survivor_submitted"
                        )
            
            logging.info(f"Loaded {len(self.stopncii_hashes)} StopNCII hashes")
        except Exception as e:
            logging.error(f"Error loading StopNCII hashes: {e}")
    
    def check_against_survivor_hashes(self, hash_hex: str, threshold: int = 5) -> bool:
        """Check if image hash matches survivor-submitted content"""
        for survivor_hash in self.stopncii_hashes:
            if HammingUtils.hamming_distance_hex(hash_hex, survivor_hash) <= threshold:
                return True
        return False

class CSAMReportingPipeline:
    """Mandatory reporting pipeline for CSAM/NCII"""
    
    def __init__(self):
        self.reports_pending = Queue()
        self.evidence_store = {}
    
    def create_ncmec_report(self, media_id: str, hash_hex: str, 
                           detection_source: str, confidence: float) -> Dict[str, Any]:
        """Create NCMEC CyberTipline report structure"""
        report = {
            'report_id': f"ncmec_{int(time.time())}_{media_id}",
            'timestamp': datetime.now().isoformat(),
            'media_id': media_id,
            'hash': hash_hex,
            'detection_method': detection_source,
            'confidence': confidence,
            'reporter_info': {
                'company': 'Content Moderation System',
                'contact': 'moderation@company.com'
            },
            'status': 'pending_review'
        }
        
        self.reports_pending.put(report)
        self.evidence_store[report['report_id']] = {
            'hash': hash_hex,
            'media_id': media_id,
            'access_log': [],
            'retention_until': datetime.now().timestamp() + (365 * 24 * 3600)  # 1 year
        }
        
        return report
    
    def create_iwf_report(self, media_id: str, hash_hex: str, 
                         detection_source: str) -> Dict[str, Any]:
        """Create IWF (Internet Watch Foundation) report structure"""
        report = {
            'report_id': f"iwf_{int(time.time())}_{media_id}",
            'timestamp': datetime.now().isoformat(),
            'media_id': media_id,
            'hash': hash_hex,
            'detection_method': detection_source,
            'reporter_info': {
                'organization': 'Content Moderation System',
                'country': 'US'
            },
            'status': 'pending_submission'
        }
        
        return report
    
    def log_evidence_access(self, report_id: str, accessor: str, purpose: str):
        """Log access to evidence with least privilege principle"""
        if report_id in self.evidence_store:
            self.evidence_store[report_id]['access_log'].append({
                'timestamp': datetime.now().isoformat(),
                'accessor': accessor,
                'purpose': purpose
            })

class SpecializedCSAMPipeline:
    """Integrated CSAM/NCII detection and reporting pipeline"""
    
    def __init__(self, store: HashStore, photodna_key: str = None, 
                 photodna_endpoint: str = None, csai_key: str = None):
        self.store = store
        self.reporting = CSAMReportingPipeline()
        self.stopncii = StopNCIIIntegration(store)
        
        self.photodna = None
        if photodna_key and photodna_endpoint:
            self.photodna = PhotoDNAIntegration(photodna_key, photodna_endpoint)
        
        self.csai_match = None
        if csai_key:
            self.csai_match = CSAIMatchIntegration(csai_key)
    
    def analyze_image(self, media_id: str, image_bytes: bytes, 
                     hash_hex: str) -> Dict[str, Any]:
        """Comprehensive CSAM/NCII analysis"""
        results = {
            'media_id': media_id,
            'hash': hash_hex,
            'detections': [],
            'action': 'allow',
            'confidence': 0.0,
            'reports_created': []
        }
        
        # Check against StopNCII survivor hashes
        if self.stopncii.check_against_survivor_hashes(hash_hex):
            results['detections'].append({
                'source': 'stopncii',
                'type': 'ncii_match',
                'confidence': 1.0
            })
            results['action'] = 'block'
            results['confidence'] = 1.0
        
        # PhotoDNA analysis
        if self.photodna:
            photodna_result = self.photodna.analyze_image(image_bytes)
            if 'IsMatch' in photodna_result and photodna_result['IsMatch']:
                results['detections'].append({
                    'source': 'photodna',
                    'type': 'csam_match',
                    'confidence': photodna_result.get('MatchConfidence', 1.0)
                })
                results['action'] = 'block'
                results['confidence'] = max(results['confidence'], 
                                          photodna_result.get('MatchConfidence', 1.0))
        
        # CSAI Match analysis
        if self.csai_match:
            csai_result = self.csai_match.analyze_image(image_bytes)
            if 'responses' in csai_result:
                for response in csai_result['responses']:
                    safe_search = response.get('safeSearchAnnotation', {})
                    if safe_search.get('adult') == 'VERY_LIKELY':
                        results['detections'].append({
                            'source': 'csai_match',
                            'type': 'adult_content',
                            'confidence': 0.9
                        })
                        results['confidence'] = max(results['confidence'], 0.9)
        
        # Create mandatory reports if CSAM detected
        if results['confidence'] > 0.8:
            ncmec_report = self.reporting.create_ncmec_report(
                media_id, hash_hex, 
                ';'.join([d['source'] for d in results['detections']]),
                results['confidence']
            )
            results['reports_created'].append(ncmec_report['report_id'])
            
            iwf_report = self.reporting.create_iwf_report(
                media_id, hash_hex,
                ';'.join([d['source'] for d in results['detections']])
            )
            results['reports_created'].append(iwf_report['report_id'])
        
        return results

print("CSAM/NCII specialized pipelines initialized successfully")

# Phase 1C: CSAM/NCII Specialized Pipelines

Integration with PhotoDNA, CSAI Match, and StopNCII for detecting abusive material.

In [ ]:
class ThreatExchangeIntegration:
    """Integration with Meta's ThreatExchange for hash sharing"""
    
    def __init__(self, access_token: str, store: HashStore):
        self.access_token = access_token
        self.store = store
        self.base_url = "https://graph.facebook.com/v18.0"
        
    def fetch_threat_indicators(self, privacy_type: str = "HAS_PRIVACY_GROUP") -> List[Dict]:
        """Fetch threat indicators from ThreatExchange"""
        url = f"{self.base_url}/threat_indicators"
        params = {
            'access_token': self.access_token,
            'limit': 1000,
            'fields': 'indicator,type,privacy_type,tags,status',
            'privacy_type': privacy_type
        }
        
        indicators = []
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            
            data = response.json()
            indicators.extend(data.get('data', []))
            
            while data.get('paging', {}).get('next'):
                response = requests.get(data['paging']['next'])
                response.raise_for_status()
                data = response.json()
                indicators.extend(data.get('data', []))
                
        except requests.RequestException as e:
            logging.error(f"Error fetching ThreatExchange indicators: {e}")
            
        return indicators
    
    def sync_pdq_hashes(self):
        """Sync PDQ hashes from ThreatExchange"""
        indicators = self.fetch_threat_indicators()
        synced_count = 0
        
        for indicator in indicators:
            if indicator.get('type') == 'HASH_PDQ' and indicator.get('status') == 'MALICIOUS':
                pdq_hash = indicator.get('indicator', '')
                if len(pdq_hash) == 64:  # Valid PDQ hash length
                    media_id = f"threatexchange_{indicator.get('id', pdq_hash[:16])}"
                    tags = indicator.get('tags', {}).get('data', [])
                    tag_names = [tag.get('text', '') for tag in tags]
                    
                    self.store.upsert_media_hash(
                        media_id=media_id,
                        hash_hex=pdq_hash,
                        quality=100,  # Assume high quality for curated hashes
                        source=f"threatexchange:{','.join(tag_names)}"
                    )
                    synced_count += 1
        
        logging.info(f"Synced {synced_count} PDQ hashes from ThreatExchange")
        return synced_count

class HMAActionEngine:
    """Hasher-Matcher-Actioner pattern implementation"""
    
    def __init__(self, store: HashStore, mih_index: MIHIndex):
        self.store = store
        self.mih_index = mih_index
        self.action_rules = {}
        self.audit_log = []
    
    def add_action_rule(self, rule_name: str, source_pattern: str, 
                       max_distance: int, action: str, priority: int = 100):
        """Add match → action rule"""
        self.action_rules[rule_name] = {
            'source_pattern': source_pattern,
            'max_distance': max_distance,
            'action': action,  # 'block', 'blur', 'queue', 'flag'
            'priority': priority
        }
    
    def process_image(self, media_id: str, image_path: str = None, 
                     image_bytes: bytes = None) -> Dict[str, Any]:
        """Process image through hash-match-action pipeline"""
        try:
            # Hash
            if image_path:
                hash_hex, quality = PDQHasher.compute_pdq(image_path)
            elif image_bytes:
                hash_hex, quality = PDQHasher.compute_pdq_from_bytes(image_bytes)
            else:
                raise ValueError("Must provide either image_path or image_bytes")
            
            # Match
            matches = []
            for rule_name, rule in self.action_rules.items():
                rule_matches = self.mih_index.query(hash_hex, rule['max_distance'])
                
                for match_id, distance in rule_matches:
                    try:
                        _, _, _, source = self.store.get_media_info(match_id)
                        if rule['source_pattern'] in source:
                            matches.append({
                                'rule': rule_name,
                                'match_id': match_id,
                                'distance': distance,
                                'action': rule['action'],
                                'priority': rule['priority'],
                                'source': source
                            })
                    except:
                        continue
            
            # Act (highest priority action wins)
            final_action = 'allow'
            if matches:
                matches.sort(key=lambda x: x['priority'])
                final_action = matches[0]['action']
            
            # Audit log
            self.audit_log.append({
                'timestamp': datetime.now().isoformat(),
                'media_id': media_id,
                'hash': hash_hex,
                'quality': quality,
                'matches': matches,
                'final_action': final_action
            })
            
            return {
                'media_id': media_id,
                'hash': hash_hex,
                'quality': quality,
                'matches': matches,
                'action': final_action
            }
            
        except Exception as e:
            logging.error(f"Error in HMA pipeline for {media_id}: {e}")
            return {
                'media_id': media_id,
                'error': str(e),
                'action': 'error'
            }

# Extend HashStore for HMA integration
def get_media_info(self, media_id: str) -> Tuple[str, int, str, str]:
    """Get full media info including source"""
    with sqlite3.connect(self.db_path) as conn:
        cursor = conn.execute('''
            SELECT hash_hex, quality, created_at, source FROM media_hashes 
            WHERE media_id = ?
        ''', (media_id,))
        row = cursor.fetchone()
        if row:
            return row[0], row[1], row[2], row[3]
        raise KeyError(f"Media ID not found: {media_id}")

HashStore.get_media_info = get_media_info

print("ThreatExchange and HMA integration initialized successfully")

# Phase 1B: Industry Hash Sources + HMA Integration

Connect to shared hash banks (GIFCT, ThreatExchange) to blockade bad content and keep lists fresh.

In [ ]:
class MIHIndex:
    """Multi-Index Hashing for fast approximate search"""
    
    def __init__(self, store: HashStore, m: int = 16):
        self.store = store
        self.m = m  # Number of chunks
        self.bits_per_chunk = 256 // m
        self.index_healthy = True
    
    def subkeys_from_hex(self, hash_hex: str) -> List[int]:
        """Extract subkeys from hash for MIH indexing"""
        hash_int = HammingUtils.hex_to_int(hash_hex)
        subkeys = []
        
        for chunk in range(self.m):
            shift = (self.m - 1 - chunk) * self.bits_per_chunk
            subkey = (hash_int >> shift) & ((1 << self.bits_per_chunk) - 1)
            subkeys.append(subkey)
        
        return subkeys
    
    def build(self):
        """Build MIH index from all stored hashes"""
        try:
            self.store.clear_mih()
            
            for media_id, hash_hex, quality in self.store.iter_all_hashes():
                subkeys = self.subkeys_from_hex(hash_hex)
                
                for chunk, subkey in enumerate(subkeys):
                    self.store.insert_mih_row(chunk, subkey, media_id)
            
            self.index_healthy = True
            logging.info(f"MIH index built successfully with {self.m} chunks")
            
        except Exception as e:
            logging.error(f"Error building MIH index: {e}")
            self.index_healthy = False
    
    def candidates(self, hash_hex: str) -> Set[str]:
        """Get candidate matches from MIH index"""
        if not self.index_healthy:
            return set()
        
        try:
            return self.store.iter_candidates_for(hash_hex, self.m, self.bits_per_chunk)
        except Exception as e:
            logging.error(f"Error querying MIH index: {e}")
            self.index_healthy = False
            return set()
    
    def query(self, hash_hex: str, max_distance: int) -> List[Tuple[str, int]]:
        """Query for similar hashes within max_distance"""
        candidates = self.candidates(hash_hex)
        results = []
        
        for media_id in candidates:
            try:
                candidate_hash, _ = self.store.get_hash(media_id)
                distance = HammingUtils.hamming_distance_hex(hash_hex, candidate_hash)
                
                if distance <= max_distance:
                    results.append((media_id, distance))
            except Exception as e:
                logging.warning(f"Error processing candidate {media_id}: {e}")
        
        results.sort(key=lambda x: (x[1], x[0]))
        return results
    
    def add_hash_with_fallback(self, media_id: str, hash_hex: str, quality: int, source: str, 
                              image_path: str = None, image_bytes: bytes = None):
        """Add hash with queue fallback if index is down"""
        try:
            self.store.upsert_media_hash(media_id, hash_hex, quality, source)
            
            if self.index_healthy:
                subkeys = self.subkeys_from_hex(hash_hex)
                for chunk, subkey in enumerate(subkeys):
                    self.store.insert_mih_row(chunk, subkey, media_id)
            else:
                self.store.queue_image(media_id, image_path, image_bytes)
                
        except Exception as e:
            logging.error(f"Error adding hash for {media_id}: {e}")
            self.store.queue_image(media_id, image_path, image_bytes)

print("MIHIndex initialized successfully")

In [ ]:
class HashStore:
    """SQLite-based storage for hashes and Multi-Index Hashing"""
    
    def __init__(self, db_path: str = "moderation.db"):
        self.db_path = db_path
        os.makedirs(os.path.dirname(db_path) if os.path.dirname(db_path) else ".", exist_ok=True)
        
    def init(self):
        """Initialize database tables"""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute('''
                CREATE TABLE IF NOT EXISTS media_hashes (
                    media_id TEXT PRIMARY KEY,
                    hash_hex TEXT NOT NULL,
                    quality INTEGER NOT NULL,
                    source TEXT NOT NULL,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            ''')
            conn.execute('''
                CREATE TABLE IF NOT EXISTS mih_index (
                    chunk INTEGER NOT NULL,
                    subkey INTEGER NOT NULL,
                    media_id TEXT NOT NULL,
                    INDEX(chunk, subkey)
                )
            ''')
            conn.execute('''
                CREATE TABLE IF NOT EXISTS hash_queue (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    media_id TEXT NOT NULL,
                    image_path TEXT,
                    image_bytes BLOB,
                    status TEXT DEFAULT 'pending',
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            ''')
            conn.commit()
    
    def upsert_media_hash(self, media_id: str, hash_hex: str, quality: int, source: str):
        """Insert or update media hash"""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute('''
                INSERT OR REPLACE INTO media_hashes 
                (media_id, hash_hex, quality, source) 
                VALUES (?, ?, ?, ?)
            ''', (media_id, hash_hex, quality, source))
            conn.commit()
    
    def iter_all_hashes(self) -> Iterator[Tuple[str, str, int]]:
        """Iterate over all stored hashes"""
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.execute('SELECT media_id, hash_hex, quality FROM media_hashes')
            for row in cursor:
                yield row
    
    def clear_mih(self):
        """Clear Multi-Index Hash table"""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute('DELETE FROM mih_index')
            conn.commit()
    
    def insert_mih_row(self, chunk: int, subkey: int, media_id: str):
        """Insert MIH index entry"""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute('''
                INSERT INTO mih_index (chunk, subkey, media_id) 
                VALUES (?, ?, ?)
            ''', (chunk, subkey, media_id))
            conn.commit()
    
    def iter_candidates_for(self, hash_hex: str, m: int, bits_per_chunk: int) -> Set[str]:
        """Find candidate matches using MIH index"""
        hash_int = HammingUtils.hex_to_int(hash_hex)
        candidates = set()
        
        with sqlite3.connect(self.db_path) as conn:
            for chunk in range(m):
                shift = (m - 1 - chunk) * bits_per_chunk
                subkey = (hash_int >> shift) & ((1 << bits_per_chunk) - 1)
                
                cursor = conn.execute('''
                    SELECT media_id FROM mih_index 
                    WHERE chunk = ? AND subkey = ?
                ''', (chunk, subkey))
                
                for row in cursor:
                    candidates.add(row[0])
        
        return candidates
    
    def get_hash(self, media_id: str) -> Tuple[str, int]:
        """Get hash and quality for media ID"""
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.execute('''
                SELECT hash_hex, quality FROM media_hashes 
                WHERE media_id = ?
            ''', (media_id,))
            row = cursor.fetchone()
            if row:
                return row[0], row[1]
            raise KeyError(f"Media ID not found: {media_id}")
    
    def queue_image(self, media_id: str, image_path: str = None, image_bytes: bytes = None):
        """Queue image for processing when index is down"""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute('''
                INSERT INTO hash_queue (media_id, image_path, image_bytes) 
                VALUES (?, ?, ?)
            ''', (media_id, image_path, image_bytes))
            conn.commit()
    
    def process_queue(self):
        """Process queued images"""
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.execute('''
                SELECT id, media_id, image_path, image_bytes FROM hash_queue 
                WHERE status = 'pending'
            ''')
            
            for queue_id, media_id, image_path, image_bytes in cursor:
                try:
                    if image_path:
                        hash_hex, quality = PDQHasher.compute_pdq(image_path)
                    elif image_bytes:
                        hash_hex, quality = PDQHasher.compute_pdq_from_bytes(image_bytes)
                    else:
                        continue
                    
                    self.upsert_media_hash(media_id, hash_hex, quality, "queued")
                    
                    conn.execute('''
                        UPDATE hash_queue SET status = 'processed' 
                        WHERE id = ?
                    ''', (queue_id,))
                    
                except Exception as e:
                    logging.error(f"Error processing queued image {media_id}: {e}")
                    conn.execute('''
                        UPDATE hash_queue SET status = 'error' 
                        WHERE id = ?
                    ''', (queue_id,))
            
            conn.commit()

print("HashStore initialized successfully")

In [ ]:
class PDQHasher:
    """Perceptual hashing using PDQ algorithm"""
    
    @staticmethod
    def compute_pdq(image_path: str) -> Tuple[str, int]:
        """Compute PDQ hash from image file"""
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Could not load image: {image_path}")
        
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        hash_vector, quality = pdqhash.compute(image_rgb)
        
        hash_int = 0
        for i, bit in enumerate(hash_vector):
            if bit:
                hash_int |= (1 << (255 - i))
        
        hash_hex = f"{hash_int:064x}"
        return hash_hex, quality
    
    @staticmethod
    def compute_pdq_from_bytes(image_bytes: bytes) -> Tuple[str, int]:
        """Compute PDQ hash from image bytes"""
        with tempfile.NamedTemporaryFile(suffix='.jpg') as tmp_file:
            tmp_file.write(image_bytes)
            tmp_file.flush()
            return PDQHasher.compute_pdq(tmp_file.name)

class HammingUtils:
    """Hamming distance utilities"""
    
    @staticmethod
    def hex_to_int(h: str) -> int:
        return int(h, 16)
    
    @staticmethod
    def hamming_distance_hex(h1: str, h2: str) -> int:
        int1 = HammingUtils.hex_to_int(h1)
        int2 = HammingUtils.hex_to_int(h2)
        return (int1 ^ int2).bit_count()

# Test PDQ hashing
print("PDQ Hasher initialized successfully")

# Phase 1A: PDQ Perceptual Hashing

PDQ turns each image into a 256-bit fingerprint for instant matching against known bad content, even after crops/edits.

In [ ]:
import os
import sys
import json
import sqlite3
import tempfile
import hashlib
import requests
import numpy as np
import cv2
from PIL import Image
from typing import Iterator, Tuple, List, Set, Optional, Dict, Any
from pathlib import Path
import logging
from datetime import datetime
import threading
from queue import Queue
import time

try:
    import pdqhash
except ImportError:
    print("Installing pdqhash...")
    os.system("pip install pdqhash")
    import pdqhash

try:
    import threatexchange
except ImportError:
    print("Installing threatexchange...")
    os.system("pip install threatexchange")
    import threatexchange

try:
    import torch
    import torchvision.transforms as transforms
    from torchvision import models
except ImportError:
    print("Installing torch and torchvision...")
    os.system("pip install torch torchvision")
    import torch
    import torchvision.transforms as transforms
    from torchvision import models

try:
    import transformers
    from transformers import CLIPProcessor, CLIPModel
except ImportError:
    print("Installing transformers...")
    os.system("pip install transformers")
    import transformers
    from transformers import CLIPProcessor, CLIPModel

try:
    import easyocr
except ImportError:
    print("Installing easyocr...")
    os.system("pip install easyocr")
    import easyocr

logging.basicConfig(level=logging.INFO)

# Image Moderation System - Phase 1 & 2 Implementation

This notebook implements a comprehensive image moderation system with:

## Phase 1 — Hash matching "seatbelts"
- **1A)** Perceptual hashing for images (PDQ)
- **1B)** Industry hash sources + HMA integration
- **1C)** CSAM/NCII specialized pipelines

## Phase 2 — Core classifiers
- **2A)** Nudity/sexual content detector
- **2B)** Violence/gore and weapons detection  
- **2C)** Hate symbols & extremist logos detection